In [63]:
import pandas as pd
import requests
import numpy as np
import json
import datetime
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', None)

def run_query(query):  # A simple function to use requests.post to make the API call.
    headers = {'X-API-KEY': 'BQYCaXaMZlqZrPCSQVsiJrKtxKRVcSe4'}
    request = requests.post('https://graphql.bitquery.io/', json={'query': query}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        print(request.reason)
        raise Exception('Query failed and return code is {}.{}'.format(request.status_code, query))

query_date = '"2022-03-16", "{}"'.format(str(datetime.date.today()))

# Convert this to JSON

VOLT_ACCOUNTS = {
    # CSP
    "SOL-P": "6Nkc8MEiz3WLz1xthYitmSuy3NGwn7782upRHo2iFmXK",
    "SOL-P-high": "JAoeEG2sfQ1xgXUNVVkJ5mCTCw4SLc6oJafDwf6jTf",
    "BTC-P": "GrB6vbG2WP7eEnbwgxUbBGRMeXYq139jo2o9oW8cNK8f",
    "SOL-P (tsUSDC)": "AQRGh6PU7LzDHvvoPNS7wVVQaCBeftw9kVDAnvuEjbs8",
    "ETH-P": "EA29Xf3HGMtYziw7UKZDUKby7gkoCbXwmiNKwc7z54Ax",
    "LUNA-P": "5kA7FPiB3t2X5s65dK1AoEu5asDjC5d7f5vaB4iY2yrj",
    "MNGO-P": "CVrRw6VtxSjokm2tKmaS5RCuoc9EFjN4wEoov6f2PST6",
#     "SOL-P-Step": ,
#     "SOL-P-Parrot": ,
#     "SOL-P-UXD": ,
    
    "SOL-C": "Hxtb6APfNtf9m8jJjh7uYp8fCTGr9aeHxBSfiPqCrV6G",
    "SOL-C-high": "DNa849drqW19uBV5X9ohpJ5brRGzq856gk3HDRqveFrA",
    "BTC-C": "DA1M8mw7GnPNKU9ReANtHPQyuVzKZtsuuSbCyc2uX2du",
    "mSOL-C": "6asST5hurmxJ8uFvh7ZRWkrMfSEzjEAJ4DNR1is3G6eH",
    "ETH-C": "FThcy5XXvab5u3jbA6NjWKdMNiCSV3oY5AAkvEvpa8wp",
    "FTT-C": "7KqHFuUksvNhrWgoacKkqyp2RwfBNdypCYgK9nxD1d6K",
    "SRM-C": "2P427N5sYcEXvZAZwqNzjXEHsBMESQoLyjNquTSmGPMb",
    "MNGO-C": "B3yakZxwomkmnCxRr8ZmQtiWgtxtVBuCREDFDdAvcCVQ",
    "scnSOL-C": "A5MpyajTy6hdsg3S2em5ukcgY1ZBhxTxEKv8BgHajv1A",
    "SBR-C": "BH7Jg3f97FyeGxsPR7FFskvfqGiaLeUnJ9Ksda53Jj8h",
    "LUNA-C": "5oV1Yf8q1oQgPYuHjepjmKFuaG2Wng9dzTqbSWhU5W2X",
    "RAY-C": "A6XsYxGj9wpqUZG81XwgQJ2zJ3efCbuWSQfnkHqUSmdM",
    "STEP-C": "FiEHDTKT6X7VFwGaUmsm1XXYr8vvkoSR5EqcY4znpefq",
    "AVAX-C": "HLfkEc6E2HEcABaWm3QZSfDvswzXjh1onF9xEzUAtWNS",
}

ACCOUNT_STRING = str(list(VOLT_ACCOUNTS.values())).replace("\'", "\"")

FEES_ADDRESS = '3KjJiWBfaw96qGhysq6Fc9FTxdPgPTNY6shM7Bwfp8EJ'

MASTER_ADDRESS = 'DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE'

query = """
    query{
    solana(network: solana) {
    transfers(
      date: {between: ["""+ query_date +"""]}
      transferType: {is: transfer}
      any: [{receiverAddress: {in: """ + ACCOUNT_STRING + """}}]
    ) {
      amount
      currency {
        symbol
        address
      }
      block {
        timestamp {
          iso8601
        }
      }
      transaction {
        signer
        signature
        innerInstructionsCount
      }
      receiver {
        address
      }
      sender {
        address
      }
    }
  }
}
"""

result = run_query(query)
# convert GraphQL json to pandas dataframe
df = pd.json_normalize(result['data']['solana']['transfers'])
df = df.rename(columns={"block.timestamp.iso8601": "date", "currency.symbol": "symbol", 
                        "transaction.signer":"signer", "currency.address":"cash",
                        "receiver.address":"receiver", "sender.address":"asset", "transaction.signature": "tx_id"})
# Does json_normalize flatten a json file for you??

df["tx_id"] = df["tx_id"].apply(lambda x: "https://solscan.io/tx/{}".format(x))
# if transaction signer & receiver is diff, drop row
df = df.query("(signer == '{}')".format(MASTER_ADDRESS)).reset_index(drop=True)

def find_key(value, d): 
    for k, v in d.items():
        if value == v:
            return k

In [64]:
df

,amount,symbol,cash,date,signer,tx_id,transaction.innerInstructionsCount,receiver,asset
0,0.000000,,3ShKthC7P2wfbzvdo8ynLjvxthTXkJSD7ZptUphSuiKQ,2022-03-18T03:30:59Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/WbescmvdrQNw6eaND32eaa287PTqoTRyFZ1fYUoLdeW49idNJpB3AUdrZtR7t6RibkQB3T7two6Vbo3uhMToM9H,10,5kA7FPiB3t2X5s65dK1AoEu5asDjC5d7f5vaB4iY2yrj,D3dxETbzyJxg33xFVSB3q5Fm1LZwfGeiA7wJQPoc5JX7
1,0.000000,,3ShKthC7P2wfbzvdo8ynLjvxthTXkJSD7ZptUphSuiKQ,2022-03-18T03:31:15Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/475GWW94R8a16vA1N2AURvBdePESCBY33eVTTNizyejyxM2Goi6iZ5WcrNx6pEZBwLHA9WQj61Cbo9WNCjDRk2F7,9,5kA7FPiB3t2X5s65dK1AoEu5asDjC5d7f5vaB4iY2yrj,D3dxETbzyJxg33xFVSB3q5Fm1LZwfGeiA7wJQPoc5JX7
2,0.000000,,4Rnv72rj6y4Jg9sXLx9oyPtgeYAsfY1UfewMEVvaaTGf,2022-03-18T03:39:54Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/2MgN1nw2JUk6uVXBRFafPPAeB5KYuqiu6kmS5AqTLkcD13KntCSo94haEgpC5uusMJwPYkBHUFZTCBerY7g7e5E4,10,2P427N5sYcEXvZAZwqNzjXEHsBMESQoLyjNquTSmGPMb,DN3nfV5L6RzbWsUa3BHa4kcoUBWnUb1Ao22JzfAHRLmf
3,0.000000,,4Rnv72rj6y4Jg9sXLx9oyPtgeYAsfY1UfewMEVvaaTGf,2022-03-18T03:40:07Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/iVPezuUW2PndTcewSenFoYtbwgU6M4eUixFGJePwF1u3F6fio6gFmHv8icQ66PQPBjcQQ8EQHFFf3KTQsov2uBw,9,2P427N5sYcEXvZAZwqNzjXEHsBMESQoLyjNquTSmGPMb,DN3nfV5L6RzbWsUa3BHa4kcoUBWnUb1Ao22JzfAHRLmf
4,0.000000,,77Q5BcUBNTSEDcLXntBaDpRCHX4NNEoDwJdqkjBaD2A3,2022-03-18T03:25:04Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/5PpTpJ6GutyA3R5qfPtogk3yegxZnYwJxQpu4Ue5b2re6CsUy1147S7W8earjd3pFeKHf3PA9mDxNnYUQmDYgyr4,10,FThcy5XXvab5u3jbA6NjWKdMNiCSV3oY5AAkvEvpa8wp,BPZ2YCtGYayEGLReNGyZReZsU5Q6faurarbM6frrFf8D
...,...,...,...,...,...,...,...,...,...
187,17154.792423,mSOL,mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So,2022-03-18T02:27:39Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/4HSro1YPguiKpTGvCxfWqgqSLFcDb5hsd1uj5pGrXoNBin5awi3yQXcAGhzD42Er91AwxSg9Byi2DKznPJmvPCuq,4,6asST5hurmxJ8uFvh7ZRWkrMfSEzjEAJ4DNR1is3G6eH,6asST5hurmxJ8uFvh7ZRWkrMfSEzjEAJ4DNR1is3G6eH
188,72.440000,scnSOL,5oVNBeEEQvYi1cX3ir8Dx5n1P7pdxydbGF2X4TxVusJm,2022-03-17T22:58:42Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/2gndTTZh8SdgjmwNmGDJiLTaiarxtciGwwVg5gxMSeaRyuJCHDvaVG4KPtPNNrtL7Ztgs5mcwH1f4BXvAoTTVug,5,A5MpyajTy6hdsg3S2em5ukcgY1ZBhxTxEKv8BgHajv1A,2nqdPZqZFac8mCdLnCkafCZ2akjE6Ljt65UMqJK9jK7g
189,0.000000,scnSOL,5oVNBeEEQvYi1cX3ir8Dx5n1P7pdxydbGF2X4TxVusJm,2022-03-17T22:58:52Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/2xMGr2MrWHtfcHm61sLMFHGLeApCLWWFDm7mAZGxj7ZZJcx5uByrxtaHhKacFFuxArY9NccMGQZ3sC4ZWECRbJYu,4,A5MpyajTy6hdsg3S2em5ukcgY1ZBhxTxEKv8BgHajv1A,2nqdPZqZFac8mCdLnCkafCZ2akjE6Ljt65UMqJK9jK7g
190,21384.000000,scnSOL,5oVNBeEEQvYi1cX3ir8Dx5n1P7pdxydbGF2X4TxVusJm,2022-03-18T02:27:03Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/2uPE9gv4YnxTaah5NQnz3CQasXcJHVwjKKWBUMZqVcruXBeWmdjaphmu8CLVNre1q2eqthUQxWnSiyiceEeYfyVA,3,A5MpyajTy6hdsg3S2em5ukcgY1ZBhxTxEKv8BgHajv1A,DP1eJCVK8fPru8H3ougLg5wzCFAYVbGKJAjGdquMJ1R3


In [65]:
df['asset'] = df.receiver.apply(lambda x: find_key(x, VOLT_ACCOUNTS))
df['symbol'] = df.apply(lambda x: "tsUSDC" if x.cash == "Cvvh8nsKZet59nsDDo3orMa3rZnPWQhpgrMCVcRDRgip" else x.symbol, axis=1)
# df["FeeType"] = df["transaction.innerInstructionsCount"].apply(lambda x: "Withdrawal Fee" if x==1 else "Performance Fee")

In [66]:
df.sort_values("date")[::-1]

,amount,symbol,cash,date,signer,tx_id,transaction.innerInstructionsCount,receiver,asset
178,1273.203514,USDC,EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,2022-03-18T03:58:06Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/66EAMx1VkQMgNYEFzbanceupKBj3CaM3bkXBo7z52NtP5bc2Nk3YXz385RkSrGnZbbMadVPzRQEKN53rHJnAiiq8,9,CVrRw6VtxSjokm2tKmaS5RCuoc9EFjN4wEoov6f2PST6,MNGO-P
41,0.000000,,zodhS2qD5QLXVzE5MNXjRHyMEb5Q5YogRficSSL76c9,2022-03-18T03:58:06Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/66EAMx1VkQMgNYEFzbanceupKBj3CaM3bkXBo7z52NtP5bc2Nk3YXz385RkSrGnZbbMadVPzRQEKN53rHJnAiiq8,9,CVrRw6VtxSjokm2tKmaS5RCuoc9EFjN4wEoov6f2PST6,MNGO-P
87,0.023358,SOL,-,2022-03-18T03:57:55Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/4QpGtY2LHcL3bdeVK3HNHTCiFiigfFLB4z6AYFRhxBf5C3k7fn6KLPssWV16h2YapULLKkvZfXf7h6e6VMiAY9g8,10,CVrRw6VtxSjokm2tKmaS5RCuoc9EFjN4wEoov6f2PST6,MNGO-P
40,0.000000,,zodhS2qD5QLXVzE5MNXjRHyMEb5Q5YogRficSSL76c9,2022-03-18T03:57:55Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/4QpGtY2LHcL3bdeVK3HNHTCiFiigfFLB4z6AYFRhxBf5C3k7fn6KLPssWV16h2YapULLKkvZfXf7h6e6VMiAY9g8,10,CVrRw6VtxSjokm2tKmaS5RCuoc9EFjN4wEoov6f2PST6,MNGO-P
177,0.000000,USDC,EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,2022-03-18T03:57:55Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/4QpGtY2LHcL3bdeVK3HNHTCiFiigfFLB4z6AYFRhxBf5C3k7fn6KLPssWV16h2YapULLKkvZfXf7h6e6VMiAY9g8,10,CVrRw6VtxSjokm2tKmaS5RCuoc9EFjN4wEoov6f2PST6,MNGO-P
...,...,...,...,...,...,...,...,...,...
99,4196.974557,USDC,EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,2022-03-17T21:36:51Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/2ydMZJdC2JKspiN4n3ZC9Az7KKmGigVHcX8jDnmHw2bR3DhjWM47EZc1W64CzuvN3bkAn1FZQQnxnBzmG3NxMYuu,1,CVrRw6VtxSjokm2tKmaS5RCuoc9EFjN4wEoov6f2PST6,MNGO-P
98,2246.950860,USDC,EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,2022-03-17T21:36:13Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/4GGasNVhDSNkfbubTDJwEwH3FMDHjLvgkDNJ5CwLKXXBE6Gh3tFFpoetHwmqWsNjstnCo5y2Zrfk3eri3mLKFNuY,1,EA29Xf3HGMtYziw7UKZDUKby7gkoCbXwmiNKwc7z54Ax,ETH-P
97,11157.435240,USDC,EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,2022-03-17T21:35:45Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/Gctk3781geboZ2BpgKUpV7B3vKdbbxSZT4dnRUwKeAB5FxPaKP7Nn38TKBmdjCv71EC98jnEt4PpXUxHNhMb6sh,1,GrB6vbG2WP7eEnbwgxUbBGRMeXYq139jo2o9oW8cNK8f,BTC-P
96,47539.016784,USDC,EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,2022-03-17T21:34:59Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/2g4UWaHoMm9MrSAPaX3Bii5pGjcAkn7PkvzSHE7vqWiC8p1EuvYkXjWPt4EwF4L2sJEAFfvMh4TpnRNnKzoSJpUM,1,6Nkc8MEiz3WLz1xthYitmSuy3NGwn7782upRHo2iFmXK,SOL-P


In [67]:
df.loc[df.symbol==""].sort_values("amount")[::-1]

,amount,symbol,cash,date,signer,tx_id,transaction.innerInstructionsCount,receiver,asset
41,0.0,,zodhS2qD5QLXVzE5MNXjRHyMEb5Q5YogRficSSL76c9,2022-03-18T03:58:06Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/66EAMx1VkQMgNYEFzbanceupKBj3CaM3bkXBo7z52NtP5bc2Nk3YXz385RkSrGnZbbMadVPzRQEKN53rHJnAiiq8,9,CVrRw6VtxSjokm2tKmaS5RCuoc9EFjN4wEoov6f2PST6,MNGO-P
9,0.0,,7dL2cJTWiMF1fWjwmH6462LJyJB5HWzMDqqWUdAh4nFz,2022-03-18T03:13:15Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/PZseHzMrZUVFCTGLSvsHLpt5yMUYGAQFMjvgpnihsGNxkJNyPx1z5apyrj69FvUdrtFBciXvg9EjThpbUVyMJ4F,9,6asST5hurmxJ8uFvh7ZRWkrMfSEzjEAJ4DNR1is3G6eH,mSOL-C
16,0.0,,AqCEkaGoiaGw19Fg1D676Bpk2oFFiCXNzXDhUnxq8AAq,2022-03-18T03:07:57Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/3c9MVuXuUujwsorR7FZWS1tgCLf1oBFBRVje42EcRdczFE781YJPrfybZ1zStD1tAWnEhqxxeaNnc1sgV7wd3aCF,10,6Nkc8MEiz3WLz1xthYitmSuy3NGwn7782upRHo2iFmXK,SOL-P
15,0.0,,8uTTV4pr3Xivhy9jgD1fqFqhBQR64mp1VTZz6oJMwvvh,2022-03-18T03:09:33Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/tePy3Sto1jAtcL8UzYXA8jDUGevAu6Q8GrhW7TDdVCp8yV6J7Y3FsdQTz3vMETtz7GSqfkir4AsMMVUy7tcqBjC,9,AQRGh6PU7LzDHvvoPNS7wVVQaCBeftw9kVDAnvuEjbs8,SOL-P (tsUSDC)
14,0.0,,8uTTV4pr3Xivhy9jgD1fqFqhBQR64mp1VTZz6oJMwvvh,2022-03-18T03:09:23Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/Kg9mMNY8TimjnLxG5zy7SXLGF7YSdh5ekudDzyR6kDTa6dgpyR4jspNZ4m6Pt39GT8ZLRrEKhZPBY78RtgvKc4r,10,AQRGh6PU7LzDHvvoPNS7wVVQaCBeftw9kVDAnvuEjbs8,SOL-P (tsUSDC)
13,0.0,,8m4xetaDDi9k5raRAg4wSdhUJRZA8dVVdiFQdj38uaMu,2022-03-18T03:41:25Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/3cVjGpsBuPupNdTYyo8UTrSN1bjDwMgACbyzaTnZuPhq5qpHQJdH4KGK3fKzmTmdVtVVLo34i9HWi2krjYsrac4s,9,A6XsYxGj9wpqUZG81XwgQJ2zJ3efCbuWSQfnkHqUSmdM,RAY-C
12,0.0,,8m4xetaDDi9k5raRAg4wSdhUJRZA8dVVdiFQdj38uaMu,2022-03-18T03:41:18Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/4JfAQDJXAK54rmz67HNNXh1ugZTNytjSHH4YrCM1dHvyh6gLTKfvXLBSBs8hvcaLCT5tfw3PQXN67XAsWW9Do3K1,10,A6XsYxGj9wpqUZG81XwgQJ2zJ3efCbuWSQfnkHqUSmdM,RAY-C
11,0.0,,7ttTkSFZTbn3idtxugeREjtMgWEEZ8H5AXE4GJNyKhya,2022-03-18T03:42:24Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/3USSMvmx8b84LcJRo4N5EpVLXEwTFJWwKveZLw7FtZDfixNkoxEMFCW7Tx5dpo8LuE8wyMju7vQaEHejCM7NgPAw,9,BH7Jg3f97FyeGxsPR7FFskvfqGiaLeUnJ9Ksda53Jj8h,SBR-C
10,0.0,,7ttTkSFZTbn3idtxugeREjtMgWEEZ8H5AXE4GJNyKhya,2022-03-18T03:42:18Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/3sfhq5vL76xuxWwMvgjQFaL3UTRp8mUvYuv1sCX4ErDL9YpHsfquq1us61Ve3A8FDG2dDLjk5cmYM4nkSMfuu7aP,10,BH7Jg3f97FyeGxsPR7FFskvfqGiaLeUnJ9Ksda53Jj8h,SBR-C
8,0.0,,7dL2cJTWiMF1fWjwmH6462LJyJB5HWzMDqqWUdAh4nFz,2022-03-18T03:10:40Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/31pJyEPLu12gW5RHAsre36zWTPk6giPvRh9w5KUMjarmbsqza5nDQsSbX51VawiGiCwbA47TpXvGMwZkqaXx7qbw,10,6asST5hurmxJ8uFvh7ZRWkrMfSEzjEAJ4DNR1is3G6eH,mSOL-C


In [68]:
df.loc[df.symbol==""].groupby("asset").amount.count()

asset
BTC-C             2
BTC-P             2
ETH-C             2
ETH-P             2
FTT-C             2
LUNA-C            3
LUNA-P            2
MNGO-C            2
MNGO-P            2
RAY-C             2
SBR-C             2
SOL-C             3
SOL-P             2
SOL-P (tsUSDC)    2
SRM-C             2
STEP-C            2
mSOL-C            2
scnSOL-C          2
Name: amount, dtype: int64

In [69]:
df.loc[df.symbol==""]

,amount,symbol,cash,date,signer,tx_id,transaction.innerInstructionsCount,receiver,asset
0,0.0,,3ShKthC7P2wfbzvdo8ynLjvxthTXkJSD7ZptUphSuiKQ,2022-03-18T03:30:59Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/WbescmvdrQNw6eaND32eaa287PTqoTRyFZ1fYUoLdeW49idNJpB3AUdrZtR7t6RibkQB3T7two6Vbo3uhMToM9H,10,5kA7FPiB3t2X5s65dK1AoEu5asDjC5d7f5vaB4iY2yrj,LUNA-P
1,0.0,,3ShKthC7P2wfbzvdo8ynLjvxthTXkJSD7ZptUphSuiKQ,2022-03-18T03:31:15Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/475GWW94R8a16vA1N2AURvBdePESCBY33eVTTNizyejyxM2Goi6iZ5WcrNx6pEZBwLHA9WQj61Cbo9WNCjDRk2F7,9,5kA7FPiB3t2X5s65dK1AoEu5asDjC5d7f5vaB4iY2yrj,LUNA-P
2,0.0,,4Rnv72rj6y4Jg9sXLx9oyPtgeYAsfY1UfewMEVvaaTGf,2022-03-18T03:39:54Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/2MgN1nw2JUk6uVXBRFafPPAeB5KYuqiu6kmS5AqTLkcD13KntCSo94haEgpC5uusMJwPYkBHUFZTCBerY7g7e5E4,10,2P427N5sYcEXvZAZwqNzjXEHsBMESQoLyjNquTSmGPMb,SRM-C
3,0.0,,4Rnv72rj6y4Jg9sXLx9oyPtgeYAsfY1UfewMEVvaaTGf,2022-03-18T03:40:07Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/iVPezuUW2PndTcewSenFoYtbwgU6M4eUixFGJePwF1u3F6fio6gFmHv8icQ66PQPBjcQQ8EQHFFf3KTQsov2uBw,9,2P427N5sYcEXvZAZwqNzjXEHsBMESQoLyjNquTSmGPMb,SRM-C
4,0.0,,77Q5BcUBNTSEDcLXntBaDpRCHX4NNEoDwJdqkjBaD2A3,2022-03-18T03:25:04Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/5PpTpJ6GutyA3R5qfPtogk3yegxZnYwJxQpu4Ue5b2re6CsUy1147S7W8earjd3pFeKHf3PA9mDxNnYUQmDYgyr4,10,FThcy5XXvab5u3jbA6NjWKdMNiCSV3oY5AAkvEvpa8wp,ETH-C
5,0.0,,77Q5BcUBNTSEDcLXntBaDpRCHX4NNEoDwJdqkjBaD2A3,2022-03-18T03:26:34Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/2vj3qGBXgYdxTUS6JkeGNSBqmF5KHNi99GXVkN2YVnYd3Qzf8F5FDLQCQ35J5TgxM6VrBw8iLV7sMptgkVmS8EMA,9,FThcy5XXvab5u3jbA6NjWKdMNiCSV3oY5AAkvEvpa8wp,ETH-C
6,0.0,,7EdWXtBq39eanGJ5JadjiqVtM12u1JZs97xQmDJMKHnC,2022-03-18T03:21:04Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/5NY9tTDVSr6BF1Kx5aQdKLpDwRP98FWUYXA46tcgRV2zRACQUJxYtB5684DuyMkcLPpHw79idJgTepaGBRWKyRfB,10,GrB6vbG2WP7eEnbwgxUbBGRMeXYq139jo2o9oW8cNK8f,BTC-P
7,0.0,,7EdWXtBq39eanGJ5JadjiqVtM12u1JZs97xQmDJMKHnC,2022-03-18T03:21:15Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/452oXQbqmjUkLdf6v5MuZGDgNbee5Czy9VXqe35j8eDNLBcoPaxzw2TcqykxRmvYNtWeH6iMSTuRFv1h82jKegGz,9,GrB6vbG2WP7eEnbwgxUbBGRMeXYq139jo2o9oW8cNK8f,BTC-P
8,0.0,,7dL2cJTWiMF1fWjwmH6462LJyJB5HWzMDqqWUdAh4nFz,2022-03-18T03:10:40Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/31pJyEPLu12gW5RHAsre36zWTPk6giPvRh9w5KUMjarmbsqza5nDQsSbX51VawiGiCwbA47TpXvGMwZkqaXx7qbw,10,6asST5hurmxJ8uFvh7ZRWkrMfSEzjEAJ4DNR1is3G6eH,mSOL-C
9,0.0,,7dL2cJTWiMF1fWjwmH6462LJyJB5HWzMDqqWUdAh4nFz,2022-03-18T03:13:15Z,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,https://solscan.io/tx/PZseHzMrZUVFCTGLSvsHLpt5yMUYGAQFMjvgpnihsGNxkJNyPx1z5apyrj69FvUdrtFBciXvg9EjThpbUVyMJ4F,9,6asST5hurmxJ8uFvh7ZRWkrMfSEzjEAJ4DNR1is3G6eH,mSOL-C
